In [1]:
import pandas as pd
import numpy as np

from sklearn import model_selection
from statistics import mean, stdev
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,f1_score,precision_score, recall_score,auc,roc_curve
import time

In [2]:
data = pd.read_csv('Backward_elimination_64features.csv', header=0, index_col=0)
#data.head()
data.shape

(19028, 64)

In [3]:
data= data.dropna()
data.shape

(19028, 64)

In [4]:
data.head(5)

,portfolio_id,product_term_credit_limit,NP,cash_intent,BCC0400,BCC0416,BCC0436,BCC0437,BCC0438,BCC0446,...,REH8227,REV5320,REV5324,REV5420,REV5627,REV7110,REV7114,REV7433,REV7447,REV7800
record_nb,,,,,,,,,,,,,,,,,,,,,
1,2019-08-01,500,0,0,2,2,0,1,1,2,...,39,10600.0,10600.0,5600.0,2813.0,76.0,76.0,100.0,0.0,2.0
2,2019-08-01,500,0,0,4,4,1,2,3,3,...,85,3250.0,3250.0,1000.0,2217.0,32.0,34.0,100.0,0.0,4.0
3,2019-08-01,500,0,0,15,15,3,6,12,12,...,36,17440.0,17440.0,1500.0,6991.0,60.0,60.0,100.0,0.0,6.0
4,2019-08-01,500,0,0,0,0,0,0,0,0,...,9996,999999997.0,999999996.0,999999997.0,999999997.0,997.0,996.0,0.0,33.0,999999997.0
5,2019-08-01,500,0,1,1,1,0,1,1,1,...,8,600.0,600.0,600.0,32.0,95.0,95.0,100.0,0.0,1.0


### Imbalanced f-test dataset

In [5]:
X = data.drop(['NP', "portfolio_id"], axis = 1)
#X = data.drop(['NP'], axis = 1)
Y = data["NP"]

In [6]:
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('XGBoost', XGBClassifier(eval_metric='mlogloss')))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'

# Create StratifiedKFold object.    
tic = time.perf_counter()
for name, model in models:
    skfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring=scoring)
    f1_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='f1')
    recall_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='recall')
    precision_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='precision')
    roc_auc_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='roc_auc')
    MLA_compare.loc[row_index,'MLA used'] = name
    MLA_compare.loc[row_index, 'Test Accuracy'] = round(mean(cv_results), 4)
    MLA_compare.loc[row_index, 'Precision'] = round(mean(precision_results),4)
    MLA_compare.loc[row_index, 'Recall'] = round(mean(recall_results),4)
    MLA_compare.loc[row_index, 'f1'] = mean(f1_results)
    MLA_compare.loc[row_index, 'roc_auc'] = mean(roc_auc_results)
    
    results.append(cv_results)
    names.append(name)
    # Print the output.
    print('List of possible accuracies for {0} is: {1}'.format(name, cv_results))
    print('List of possible Precision for {0} is: {1}'.format(name, precision_results))
    print('List of possible Recall for {0} is: {1}'.format(name, recall_results))
    print('List of possible F1 score for {0} is: {1}'.format(name, f1_results))
    print('List of possible ROC_AUC for {0} is: {1}'.format(name, roc_auc_results))
    msg = "%s: %f (%f)" % (name, mean(cv_results), cv_results.std())
    print(msg)
    toc = time.perf_counter()
    secs = toc - tic
    print("---Classifier %s use %0.4f seconds ---" %(name, secs))
    row_index+=1
    
MLA_compare.sort_values(by = ['Test Accuracy'], ascending = False, inplace = True)    
MLA_compare

C:\Users\megan\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\megan\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

List of possible accuracies for Logistic Regression is: [0.21702575 0.90935365 0.17235943 0.16057819 0.16057819]
List of possible Precision for Logistic Regression is: [0.0606869  0.         0.05798928 0.05991353 0.05991353]
List of possible Recall for Logistic Regression is: [0.52753623 0.         0.53333333 0.56395349 0.56395349]
List of possible F1 score for Logistic Regression is: [0.10885167 0.         0.10460489 0.10831937 0.10831937]
List of possible ROC_AUC for Logistic Regression is: [0.31556809 0.56736178 0.28728314 0.29353116 0.28473967]
Logistic Regression: 0.323979 (0.293425)
---Classifier Logistic Regression use 4.7865 seconds ---


C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

List of possible accuracies for XGBoost is: [0.90620074 0.90541251 0.90567525 0.90407359 0.90749014]
List of possible Precision for XGBoost is: [0.38888889 0.30769231 0.34782609 0.31578947 0.42592593]
List of possible Recall for XGBoost is: [0.06086957 0.03478261 0.04637681 0.05232558 0.06686047]
List of possible F1 score for XGBoost is: [0.10526316 0.0625     0.08184143 0.08977556 0.11557789]
List of possible ROC_AUC for XGBoost is: [0.72640353 0.75270237 0.77238839 0.73509429 0.76456176]
XGBoost: 0.905770 (0.001110)
---Classifier XGBoost use 38.8069 seconds ---


,MLA used,Test Accuracy,Precision,Recall,f1,roc_auc
1,XGBoost,0.9058,0.3572,0.0522,0.090992,0.750230
0,Logistic Regression,0.3240,0.0477,0.4378,0.086019,0.349697


## Balanced dataset -- Down Sampling the Majority Class

In [7]:
from imblearn.under_sampling import RandomUnderSampler

X = data.drop(['NP', "portfolio_id"], axis = 1)
y = data["NP"]
rus = RandomUnderSampler(replacement=False)
X_subsample, y_subsample = rus.fit_resample(X, y)
print(X.shape)
print(X_subsample.shape)

(19028, 62)
(3446, 62)


In [8]:
#X_subsample.head()

In [9]:
X = X_subsample
#X = data.drop(['NP'], axis = 1)

Y = y_subsample

In [10]:
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('XGBoost', XGBClassifier(eval_metric='mlogloss')))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'

# Create StratifiedKFold object.    
tic = time.perf_counter()
for name, model in models:
    skfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring=scoring)
    f1_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='f1')
    recall_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='recall')
    precision_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='precision')
    roc_auc_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='roc_auc')
    MLA_compare.loc[row_index,'MLA used'] = name
    MLA_compare.loc[row_index, 'Test Accuracy'] = round(mean(cv_results), 4)
    MLA_compare.loc[row_index, 'Precision'] = round(mean(precision_results),4)
    MLA_compare.loc[row_index, 'Recall'] = round(mean(recall_results),4)
    MLA_compare.loc[row_index, 'f1'] = mean(f1_results)
    MLA_compare.loc[row_index, 'roc_auc'] = mean(roc_auc_results)
    
    results.append(cv_results)
    names.append(name)
    # Print the output.
    print('List of possible accuracies for {0} is: {1}'.format(name, cv_results))
    print('List of possible Precision for {0} is: {1}'.format(name, precision_results))
    print('List of possible Recall for {0} is: {1}'.format(name, recall_results))
    print('List of possible F1 score for {0} is: {1}'.format(name, f1_results))
    print('List of possible ROC_AUC for {0} is: {1}'.format(name, roc_auc_results))
    msg = "%s: %f (%f)" % (name, mean(cv_results), cv_results.std())
    print(msg)
    toc = time.perf_counter()
    secs = toc - tic
    print("---Classifier %s use %0.4f seconds ---" %(name, secs))
    row_index+=1
    
MLA_compare.sort_values(by = ['Test Accuracy'], ascending = False, inplace = True)    
MLA_compare

List of possible accuracies for Logistic Regression is: [0.66376812 0.66908563 0.65166909 0.63425254 0.66328012]
List of possible Precision for Logistic Regression is: [0.75565611 0.79292929 0.71848739 0.68379447 0.73443983]
List of possible Recall for Logistic Regression is: [0.48405797 0.45639535 0.49709302 0.50144928 0.51304348]
List of possible F1 score for Logistic Regression is: [0.59010601 0.57933579 0.58762887 0.57859532 0.60409556]
List of possible ROC_AUC for Logistic Regression is: [0.70981727 0.71517947 0.70042973 0.68864594 0.70955089]
Logistic Regression: 0.656411 (0.012453)
---Classifier Logistic Regression use 0.4691 seconds ---


C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

List of possible accuracies for XGBoost is: [0.67246377 0.68795356 0.70682148 0.68650218 0.66618287]
List of possible Precision for XGBoost is: [0.67975831 0.69724771 0.71779141 0.69724771 0.66475645]
List of possible Recall for XGBoost is: [0.65217391 0.6627907  0.68023256 0.66086957 0.67246377]
List of possible F1 score for XGBoost is: [0.66568047 0.67958271 0.69850746 0.67857143 0.6685879 ]
List of possible ROC_AUC for XGBoost is: [0.73597984 0.74806623 0.76768622 0.75120913 0.71664981]
XGBoost: 0.683985 (0.014094)
---Classifier XGBoost use 6.4529 seconds ---


,MLA used,Test Accuracy,Precision,Recall,f1,roc_auc
1,XGBoost,0.6840,0.6914,0.6657,0.678186,0.743918
0,Logistic Regression,0.6564,0.7371,0.4904,0.587952,0.704725


## Balanced dataset -- Up Sampling the Minority Class

### Logistic Regression for up sampling

In [11]:
from imblearn.over_sampling import RandomOverSampler

X = data.drop(['NP', "portfolio_id"], axis = 1)
y = data["NP"]
ros = RandomOverSampler()
X_oversample, y_oversample = ros.fit_resample(X, y)
print(X.shape)
print(X_oversample.shape)

(19028, 62)
(34610, 62)


In [12]:
X = X_oversample
#X = data.drop(['NP'], axis = 1)

Y = y_oversample

In [13]:
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('XGBoost', XGBClassifier(eval_metric='mlogloss')))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'

# Create StratifiedKFold object.    
tic = time.perf_counter()
for name, model in models:
    skfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring=scoring)
    f1_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='f1')
    recall_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='recall')
    precision_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='precision')
    roc_auc_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='roc_auc')
    MLA_compare.loc[row_index,'MLA used'] = name
    MLA_compare.loc[row_index, 'Test Accuracy'] = round(mean(cv_results), 4)
    MLA_compare.loc[row_index, 'Precision'] = round(mean(precision_results),4)
    MLA_compare.loc[row_index, 'Recall'] = round(mean(recall_results),4)
    MLA_compare.loc[row_index, 'f1'] = mean(f1_results)
    MLA_compare.loc[row_index, 'roc_auc'] = mean(roc_auc_results)
    
    results.append(cv_results)
    names.append(name)
    # Print the output.
    print('List of possible accuracies for {0} is: {1}'.format(name, cv_results))
    print('List of possible Precision for {0} is: {1}'.format(name, precision_results))
    print('List of possible Recall for {0} is: {1}'.format(name, recall_results))
    print('List of possible F1 score for {0} is: {1}'.format(name, f1_results))
    print('List of possible ROC_AUC for {0} is: {1}'.format(name, roc_auc_results))
    msg = "%s: %f (%f)" % (name, mean(cv_results), cv_results.std())
    print(msg)
    toc = time.perf_counter()
    secs = toc - tic
    print("---Classifier %s use %0.4f seconds ---" %(name, secs))
    row_index+=1
    
MLA_compare.sort_values(by = ['Test Accuracy'], ascending = False, inplace = True)    
MLA_compare

List of possible accuracies for Logistic Regression is: [0.4995666  0.66671482 0.49971107 0.49971107 0.4986998 ]
List of possible Precision for Logistic Regression is: [0.49978251 0.77089202 0.49985545 0.49985537 0.49934905]
List of possible Recall for Logistic Regression is: [0.99595493 0.47442936 0.9991332  0.99855533 0.9973996 ]
List of possible F1 score for Logistic Regression is: [0.6655725  0.58737256 0.66634551 0.66621687 0.66550993]
List of possible ROC_AUC for Logistic Regression is: [0.60913902 0.71410686 0.60656366 0.61899103 0.60726662]
Logistic Regression: 0.532881 (0.066918)
---Classifier Logistic Regression use 2.7271 seconds ---


C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

List of possible accuracies for XGBoost is: [0.93282288 0.92603294 0.92429934 0.93036695 0.92834441]
List of possible Precision for XGBoost is: [0.88331627 0.87817389 0.86999244 0.88162952 0.88159588]
List of possible Recall for XGBoost is: [0.9973996  0.98930945 0.99768853 0.99422132 0.98959838]
List of possible F1 score for XGBoost is: [0.93689782 0.93043478 0.9294751  0.93454644 0.93248026]
List of possible ROC_AUC for XGBoost is: [0.97051084 0.96710621 0.96977344 0.97321639 0.96961269]
XGBoost: 0.928373 (0.003028)
---Classifier XGBoost use 32.0343 seconds ---


,MLA used,Test Accuracy,Precision,Recall,f1,roc_auc
1,XGBoost,0.9284,0.8789,0.9936,0.932767,0.970044
0,Logistic Regression,0.5329,0.5539,0.8931,0.650203,0.631213


### SMOTE for imbalanced dataset

In [14]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [15]:
from sklearn.datasets import fetch_openml

data = pd.read_csv('Backward_elimination_64features.csv', header=0, index_col=0)
data = data.dropna()

In [16]:
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from sklearn.model_selection import cross_validate
import numpy as np

X = data.drop(['NP', "portfolio_id"], axis = 1)
y = data["NP"]
# for imblearn we should have these classes
y = (y.astype(np.int) + 1) // 2

C:\Users\megan\AppData\Local\Temp/ipykernel_20044/1056751668.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = (y.astype(np.int) + 1) // 2


In [17]:
X_synthetic_sample, y_synthetic_sample = smote.fit_resample(X, y)

X_synthetic_sample.head()

,product_term_credit_limit,cash_intent,BCC0400,BCC0416,BCC0436,BCC0437,BCC0438,BCC0446,BCC0700,BCC1300,...,REH8227,REV5320,REV5324,REV5420,REV5627,REV7110,REV7114,REV7433,REV7447,REV7800
0,500,0,2,2,0,1,1,2,1,3,...,39,10600.0,10600.0,5600.0,2813.0,76.0,76.0,100.0,0.0,2.0
1,500,0,4,4,1,2,3,3,3,4,...,85,3250.0,3250.0,1000.0,2217.0,32.0,34.0,100.0,0.0,4.0
2,500,0,15,15,3,6,12,12,0,15,...,36,17440.0,17440.0,1500.0,6991.0,60.0,60.0,100.0,0.0,6.0
3,500,0,0,0,0,0,0,0,10,0,...,9996,999999997.0,999999996.0,999999997.0,999999997.0,997.0,996.0,0.0,33.0,999999997.0
4,500,1,1,1,0,1,1,1,3,1,...,8,600.0,600.0,600.0,32.0,95.0,95.0,100.0,0.0,1.0


In [18]:
X = X_synthetic_sample
#X = data.drop(['NP'], axis = 1)

Y = y_synthetic_sample

In [19]:
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('XGBoost', XGBClassifier(eval_metric='mlogloss')))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'

# Create StratifiedKFold object.    
tic = time.perf_counter()
for name, model in models:
    skfold = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring=scoring)
    f1_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='f1')
    recall_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='recall')
    precision_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='precision')
    roc_auc_results = model_selection.cross_val_score(model, X, Y, cv=skfold, scoring='roc_auc')
    MLA_compare.loc[row_index,'MLA used'] = name
    MLA_compare.loc[row_index, 'Test Accuracy'] = round(mean(cv_results), 4)
    MLA_compare.loc[row_index, 'Precision'] = round(mean(precision_results),4)
    MLA_compare.loc[row_index, 'Recall'] = round(mean(recall_results),4)
    MLA_compare.loc[row_index, 'f1'] = mean(f1_results)
    MLA_compare.loc[row_index, 'roc_auc'] = mean(roc_auc_results)
    
    results.append(cv_results)
    names.append(name)
    # Print the output.
    print('List of possible accuracies for {0} is: {1}'.format(name, cv_results))
    print('List of possible Precision for {0} is: {1}'.format(name, precision_results))
    print('List of possible Recall for {0} is: {1}'.format(name, recall_results))
    print('List of possible F1 score for {0} is: {1}'.format(name, f1_results))
    print('List of possible ROC_AUC for {0} is: {1}'.format(name, roc_auc_results))
    msg = "%s: %f (%f)" % (name, mean(cv_results), cv_results.std())
    print(msg)
    toc = time.perf_counter()
    secs = toc - tic
    print("---Classifier %s use %0.4f seconds ---" %(name, secs))
    row_index+=1
    
MLA_compare.sort_values(by = ['Test Accuracy'], ascending = False, inplace = True)    
MLA_compare

List of possible accuracies for Logistic Regression is: [0.50014447 0.5        0.5        0.50014447 0.49985553]
List of possible Precision for Logistic Regression is: [0.50007237 0.5        0.5        0.50007229 0.49992776]
List of possible Recall for Logistic Regression is: [0.9982664  0.99884426 0.99971107 0.99942213 0.99971107]
List of possible F1 score for Logistic Regression is: [0.66634523 0.66640964 0.66660245 0.66660243 0.66653824]
List of possible ROC_AUC for Logistic Regression is: [0.61558197 0.60955351 0.61410983 0.61275207 0.6066011 ]
Logistic Regression: 0.500029 (0.000108)
---Classifier Logistic Regression use 2.2008 seconds ---


C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\megan\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

List of possible accuracies for XGBoost is: [0.91895406 0.91317538 0.91317538 0.91577579 0.92141   ]
List of possible Precision for XGBoost is: [0.90776153 0.90625    0.89921831 0.91067352 0.90957596]
List of possible Recall for XGBoost is: [0.93267842 0.92169893 0.93065588 0.92198786 0.93585669]
List of possible F1 score for XGBoost is: [0.9200513  0.91390918 0.91466705 0.91629576 0.92252919]
List of possible ROC_AUC for XGBoost is: [0.97328372 0.97022813 0.97087858 0.96995848 0.9728876 ]
XGBoost: 0.916498 (0.003248)
---Classifier XGBoost use 38.2806 seconds ---


,MLA used,Test Accuracy,Precision,Recall,f1,roc_auc
1,XGBoost,0.9165,0.9067,0.9286,0.91749,0.971447
0,Logistic Regression,0.5000,0.5000,0.9992,0.66650,0.611720
